<a href="https://colab.research.google.com/github/kimhwijin/HandsOnMachineLearing/blob/main/Data_API_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot
import numpy as np
import os

# 데이터셋 API

In [20]:
X = tf.range(10)
dataset = tf.data.Dataset.from_tensor_slices(X)
#dataset = tf.data.Dataset.range(10)
print(X, dataset)
for item in dataset:
    print(item)

tf.Tensor([0 1 2 3 4 5 6 7 8 9], shape=(10,), dtype=int32) <TensorSliceDataset shapes: (), types: tf.int32>
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


In [21]:
#연쇄 변환
dataset = dataset.repeat(3).batch(7)
for item in dataset:
    print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)
tf.Tensor([8 9], shape=(2,), dtype=int32)


In [22]:
#데이터셋 할당받지않으면 데이터가 다 날라감
dataset.repeat(3).batch(7)
print(dataset)
dataset = tf.data.Dataset.from_tensor_slices(X).repeat(3).batch(7)

<BatchDataset shapes: (None,), types: tf.int32>


In [23]:
dataset = dataset.map(lambda x: x * 2)
for item in dataset:
    print(item)

dataset = dataset.unbatch()
dataset = dataset.filter(lambda x: x < 10)

for item in dataset.take(3):
    print(item)

tf.Tensor([ 0  2  4  6  8 10 12], shape=(7,), dtype=int32)
tf.Tensor([14 16 18  0  2  4  6], shape=(7,), dtype=int32)
tf.Tensor([ 8 10 12 14 16 18  0], shape=(7,), dtype=int32)
tf.Tensor([ 2  4  6  8 10 12 14], shape=(7,), dtype=int32)
tf.Tensor([16 18], shape=(2,), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)


In [24]:
tf.random.set_seed(42)

dataset = tf.data.Dataset.range(10).repeat(3)
dataset = dataset.shuffle(buffer_size=3, seed=42).batch(7)
for item in dataset:
    print(item)

tf.Tensor([1 3 0 4 2 5 6], shape=(7,), dtype=int64)
tf.Tensor([8 7 1 0 3 2 5], shape=(7,), dtype=int64)
tf.Tensor([4 6 9 8 9 7 0], shape=(7,), dtype=int64)
tf.Tensor([3 1 4 5 2 8 7], shape=(7,), dtype=int64)
tf.Tensor([6 9], shape=(2,), dtype=int64)


In [25]:
#셔플 후 반복
dataset = tf.data.Dataset.range(10).shuffle(buffer_size=3)
for item in dataset:
    print(item)
print("---")
dataset = dataset.repeat(3)
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
---
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(8, shape=(), d

# Split_Dataset

In [26]:
#데이터셋을 여러파일로 나누기
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#데이터셋 불러오기
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

#정규화
scaler = StandardScaler()
scaler.fit(X_train)
X_mean = scaler.mean_
X_std = scaler.scale_

In [27]:
#
def save_to_multiple_csv_files(data, name_prefix, header=None, n_parts=10):
    housing_dir = os.path.join("datasets", "housing")
    os.makedirs(housing_dir, exist_ok=True)
    path_format = os.path.join(housing_dir, "my_{}_{:02d}.csv")

    filepaths = []
    #데이터 개수를, 파일 개수로 split
    m = len(data)
    for file_idx, row_indices in enumerate(np.array_split(np.arange(m), n_parts)):
        part_csv = path_format.format(name_prefix, file_idx)
        filepaths.append(part_csv)
        #파일생성
        with open(part_csv, "wt", encoding="utf-8") as f:
            #헤더 작성
            if header is not None:
                f.write(header)
                f.write("\n")
            #데이터 작성
            for row_idx in row_indices:
                f.write(",".join([repr(col) for col in data[row_idx]]))
                f.write("\n")
    return filepaths

train_data = np.c_[X_train, y_train]
valid_data = np.c_[X_valid, y_valid]
test_data = np.c_[X_test, y_test]
header_cols = housing.feature_names + ["MedianHouseValue"]
header = ",".join(header_cols)

train_filepaths = save_to_multiple_csv_files(train_data, "train", header, n_parts=20)
valid_filepaths = save_to_multiple_csv_files(valid_data, "valid", header, n_parts=10)
test_filepaths = save_to_multiple_csv_files(test_data, "test", header, n_parts=10)

In [28]:
import pandas as pd

print(pd.read_csv(train_filepaths[0]).head())

with open(train_filepaths[0]) as f:
    for i in range(5):
        print(f.readline(), end="")

   MedInc  HouseAge  AveRooms  ...  Latitude  Longitude  MedianHouseValue
0  3.5214      15.0  3.049945  ...     37.63    -122.43             1.442
1  5.3275       5.0  6.490060  ...     33.69    -117.39             1.687
2  3.1000      29.0  7.542373  ...     38.44    -122.98             1.621
3  7.1736      12.0  6.289003  ...     33.55    -117.70             2.621
4  2.0549      13.0  5.312457  ...     33.93    -116.93             0.956

[5 rows x 9 columns]
MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHouseValue
3.5214,15.0,3.0499445061043287,1.106548279689234,1447.0,1.6059933407325193,37.63,-122.43,1.442
5.3275,5.0,6.490059642147117,0.9910536779324056,3464.0,3.4433399602385686,33.69,-117.39,1.687
3.1,29.0,7.5423728813559325,1.5915254237288134,1328.0,2.2508474576271187,38.44,-122.98,1.621
7.1736,12.0,6.289002557544757,0.9974424552429667,1054.0,2.6956521739130435,33.55,-117.7,2.621


In [29]:
#파일 경로에서 데이터셋 생성하기
filepath_dataset = tf.data.Dataset.list_files(train_filepaths, seed=42)

#한번에 5개 데이터를 읽는 데이터셋, 맨처음 줄은 열이기 때문에 스킵한다.
n_readers = 5
dataset = filepath_dataset.interleave(
    lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
    cycle_length=n_readers)

for line in dataset.take(5):
    print(line.numpy())

b'4.7361,7.0,7.464968152866242,1.1178343949044587,846.0,2.694267515923567,34.49,-117.27,1.745'
b'3.6641,17.0,5.577142857142857,1.1542857142857144,511.0,2.92,40.85,-121.07,0.808'
b'4.5909,16.0,5.475877192982456,1.0964912280701755,1357.0,2.9758771929824563,33.63,-117.71,2.418'
b'3.6875,44.0,4.524475524475524,0.993006993006993,457.0,3.195804195804196,34.04,-118.15,1.625'
b'2.3,25.0,5.828178694158075,0.9587628865979382,909.0,3.1237113402061856,36.25,-119.4,1.328'


# Preprocessing

In [30]:
n_inputs = 8 # X_train.shape[-1]

@tf.function
def preprocess(line):
    #컬럼 기본값
    defs = [0.] * n_inputs + [tf.constant([], dtype=tf.float32)]
    #컬럼당 1개씩 1차원 백터를 만든다.
    fields = tf.io.decode_csv(line, record_defaults=defs)
    #레이블 제외한 컬럼들을 뭉침
    x = tf.stack(fields[:-1])
    #레이블 컬럼
    y = tf.stack(fields[-1:])
    return (x - X_mean) / X_std, y

# DataPipeline

In [32]:
def csv_reader_dataset(filepaths, repeat=1, n_readers=5,
                       n_read_threads=None, shuffle_buffer_size=10000,
                       n_parse_threads=5, batch_size=32):
    #파일리스트
    dataset = tf.data.Dataset.list_files(filepaths).repeat(repeat)
    #n개의 TextLineDataset를 생성함
    dataset = dataset.interleave(
        lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
        cycle_length=n_readers, num_parallel_calls=n_read_threads)
    #데이터셋 셔플
    dataset = dataset.shuffle(shuffle_buffer_size)
    #csv파일 decode 및 정규화
    dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
    #배치
    dataset = dataset.batch(batch_size)
    #프리패치
    return dataset.prefetch(1)


tf.random.set_seed(42)
#example
train_set = csv_reader_dataset(train_filepaths, batch_size=3)
for X_batch, y_batch in train_set.take(2):
    print("X =", X_batch)
    print("y =", y_batch)
    print()

[<tf.Tensor 'DecodeCSV:0' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:1' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:2' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:3' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:4' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:5' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:6' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:7' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:8' shape=() dtype=float32>]
Tensor("stack:0", shape=(8,), dtype=float32)
Tensor("stack_1:0", shape=(1,), dtype=float32)
X = tf.Tensor(
[[ 0.5804519  -0.20762321  0.05616303 -0.15191229  0.01343246  0.00604472
   1.2525111  -1.3671792 ]
 [ 5.818099    1.8491895   1.1784915   0.28173092 -1.2496178  -0.3571987
   0.7231292  -1.0023477 ]
 [-0.9253566   0.5834586  -0.7807257  -0.28213993 -0.36530012  0.27389365
  -0.76194876  0.72684526]], shape=(3, 8), dtype=float32)
y = tf.Tensor(
[[1.752]
 [1.313]
 [1.535]], shape=(3, 1), dtype=float32)

X = tf.Tensor(
[[-0.8324941   0.662

# Dataset_Train

In [34]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)


batch_size = 32

train_set = csv_reader_dataset(train_filepaths, repeat=None)
valid_set = csv_reader_dataset(valid_filepaths)
test_set = csv_reader_dataset(test_filepaths)

model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1),
])

model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))

model.fit(train_set, steps_per_epoch=len(X_train) // batch_size, epochs=10,
          validation_data=valid_set)

Epoch 1/10
362/362 [==============================] - 2s 3ms/step - loss: 1.7443 - val_loss: 1.5302
Epoch 2/10
362/362 [==============================] - 1s 3ms/step - loss: 0.8780 - val_loss: 1.0380
Epoch 3/10
362/362 [==============================] - 1s 3ms/step - loss: 0.7399 - val_loss: 0.7188
Epoch 4/10
362/362 [==============================] - 1s 3ms/step - loss: 0.7234 - val_loss: 0.6914
Epoch 5/10
362/362 [==============================] - 1s 3ms/step - loss: 0.6817 - val_loss: 0.8448
Epoch 6/10
362/362 [==============================] - 1s 3ms/step - loss: 0.6367 - val_loss: 0.9235
Epoch 7/10
362/362 [==============================] - 1s 3ms/step - loss: 0.6032 - val_loss: 0.6490
Epoch 8/10
362/362 [==============================] - 1s 3ms/step - loss: 0.5945 - val_loss: 0.5858
Epoch 9/10
362/362 [==============================] - 1s 3ms/step - loss: 0.5454 - val_loss: 0.5139
Epoch 10/10
362/362 [==============================] - 1s 3ms/step - loss: 0.5092 - val_loss: 0.5005

In [43]:
model.evaluate(test_set, steps=len(X_test) // batch_size)

#batch_size = 32 이고, 3개를 꺼내니깐 총 96개의 데이터가 나옴
new_set = test_set.take(3).map(lambda X, y: X)
print(model.predict(new_set, steps=3)[:10], "...")

161/161 [==============================] - 0s 2ms/step - loss: 0.5094
[[0.8494782]
 [2.728919 ]
 [2.8378737]
 [2.009118 ]
 [4.2545223]
 [6.9662395]
 [2.3816266]
 [1.7895379]
 [3.03715  ]
 [1.1908183]] ...
